# LLM Task Optimization
This notebook demonstrates how prompt engineering techniques improve the performance of a Large Language Model (LLM).


In [1]:
# Run this in a Jupyter cell with ! or in your terminal
!pip install transformers torch


In [2]:
# 1. Import libraries
from transformers import pipeline

# 2. Load a pre-trained LLM (you can swap models depending on task complexity)
# For text generation / reasoning tasks:
generator = pipeline("text-generation", model="gpt2")

# For instruction-following tasks (better for prompt engineering):
# Try a model like "google/flan-t5-large" or "facebook/bart-large"
qa_model = pipeline("text2text-generation", model="google/flan-t5-large")


Device set to use cpu


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

C:\Users\marky\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\marky\.cache\huggingface\hub\models--google--flan-t5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [3]:
# Example: Baseline Test

In [4]:
# Input text (unstructured review)
input_text = "I bought the Samsung Galaxy S23 last week for $799 at Best Buy. The purchase date was November 15, 2025."

# Baseline prompt
baseline_prompt = "Extract the data."

baseline_output = qa_model(baseline_prompt + "\n\n" + input_text, max_length=100)
print("Baseline Output:", baseline_output[0]['generated_text'])


Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Baseline Output: Best Buy, store, Best Buy; Samsung Galaxy S23, price, "799 at Best Buy"; Samsung Galaxy S23, purchaseDate, "November 15, 2025"


# Iterative Prompt Engineering

## Technique 1: Role Prompting

In [5]:
role_prompt = """You are a Senior Data Analyst. 
Extract the product name, price, and purchase date from the following review."""

role_output = qa_model(role_prompt + "\n\n" + input_text, max_length=100)
print("Role Prompting Output:", role_output[0]['generated_text'])


Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Role Prompting Output: Samsung Galaxy S23, price, Best Buy, November 15, 2025


# Output Formatting

In [6]:
format_prompt = """Extract the product name, price, and purchase date from the following review. 
Provide the output strictly in JSON format with keys: Name, Price, Date."""

format_output = qa_model(format_prompt + "\n\n" + input_text, max_length=100)
print("Output Formatting:", format_output[0]['generated_text'])


Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Output Formatting: Name[Samsung Galaxy S23], Price[799], PurchaseDate[November 15, 2025]


# Chain-of-Thought

In [7]:
cot_prompt = """First, list the steps you will take to extract the product name, price, and purchase date. 
Then provide the final answer in JSON format with keys: Name, Price, Date."""

cot_output = qa_model(cot_prompt + "\n\n" + input_text, max_length=150)
print("Chain-of-Thought Output:", cot_output[0]['generated_text'])


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Chain-of-Thought Output: Name[Samsung Galaxy S23], Price[799], PurchaseDate[November 15, 2025]


# Final Optimized Prompt

In [8]:
final_prompt = """You are a Senior Data Analyst. Carefully extract the product name, price, and purchase date from the following review. 
Before answering, think step by step to ensure accuracy. 
Provide the final output strictly in JSON format with keys: Name, Price, Date. 
Normalize the price as a number (no currency symbol) and format the date in ISO (YYYY-MM-DD)."""

final_output = qa_model(final_prompt + "\n\n" + input_text, max_length=150)
print("Final Optimized Output:", final_output[0]['generated_text'])


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Final Optimized Output: Samsung Galaxy S23, Price, PurchaseDate, "November 15, 2025"
